# Ocean-POL

The Ocean-Pol research radar is a C-band polarimetric weather radar mounted on the CSIRO operated Research Vessel (RV) Investigator. This ship-borne radar has operated since 2017.

Data collected includes routine observations on all research voyages, including multiple trips to Antarctic waters, Heard Island and circumnavigation of Australia. Data has also been collected during field campaigns, including CAPRICORN (2016), Reef to Rainforest (2017) and Years of the Maritime Continent (2019). 

![opol](img/opol.jpg)


In [1]:
%matplotlib inline
import os
import datetime
import warnings

import pyproj
import requests
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import LogNorm

In [2]:
import bokeh
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.tile_providers import CARTODBPOSITRON_RETINA, get_provider
from bokeh.models import TapTool

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
warnings.simplefilter("ignore")
sns.set_style("whitegrid")

In [5]:
# Loading the lat/lon position of OceanPOL.
opol_position = xr.open_dataset("data/opol_position.nc")
df = opol_position.to_dataframe()
proj = pyproj.Proj("epsg:3857")
x, y = proj(opol_position.lon.values, opol_position.lat.values)
df["x"] = x
df["y"] = y
ndf = df.resample("6H").mean().dropna()
datestr = lambda x: x.strftime("%Y-%m-%d")
ndf["datestr"] = [*map(datestr, ndf.index)]
source = ColumnDataSource(ndf)

In [6]:
figure_shared_arguments = {
    "plot_width": 800,
    "plot_height": 725,
    "x_range": (11125637, 17897168),
    "y_range": (-9933422, -1133043),
    "x_axis_type": "mercator",
    "y_axis_type": "mercator",
    "tools": "pan,wheel_zoom,reset,save,tap",
}

hover = HoverTool(
    tooltips=[
        ("Date", "@datestr"),
        ("Longitude", "@lon"),
        ("Latitude", "@lat"),        
    ]
)

tile_provider = get_provider(CARTODBPOSITRON_RETINA)

map_calib = figure(title="OPOL position", **figure_shared_arguments)
map_calib.add_tile(tile_provider)
map_calib.circle(source=source, x="x", y="y")
map_calib.add_tools(hover)

taptool = map_calib.select(type=TapTool)
# taptool.callback = OpenURL(url="@name")
show(map_calib)